In [5]:
import pandas as pd
import yfinance as yf
import duckdb

In [ ]:
class YahooClient:
    def __init__(self, ticker: str):
        self.ticker = ticker
        self._yf = yf.Ticker(ticker)
    

    def identification(self):
        isin = self._yf.get_isin()
        ticker = self.ticker
        return {"ticker": ticker, "isin": isin}
    
    def dividends(self):
        return self._yf.get_dividends()
    
    def shares_outstanding(self):
        return self._yf.get_shares_outstanding()
    
    def income_statement(self):
        return self._yf.get_income_stmt()
    
    def balance_sheet(self):
        return self._yf.get_balance_sheet()

    def cashflow(self):
        return self._yf.get_cashflow()
    

In [ ]:
class 

In [20]:
YahooClient("AAPL").income_statement()

,2025-09-30,2024-09-30,2023-09-30,2022-09-30,2021-09-30
TaxEffectOfUnusualItems,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN
TaxRateForCalcs,1.560000e-01,2.410000e-01,1.470000e-01,1.620000e-01,NaN
NormalizedEBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
NetIncomeFromContinuingOperationNetMinorityInterest,1.120100e+11,9.373600e+10,9.699500e+10,9.980300e+10,NaN
ReconciledDepreciation,1.169800e+10,1.144500e+10,1.151900e+10,1.110400e+10,NaN
ReconciledCostOfRevenue,2.209600e+11,2.103520e+11,2.141370e+11,2.235460e+11,NaN
EBITDA,1.447480e+11,1.346610e+11,1.258200e+11,1.305410e+11,NaN
EBIT,1.330500e+11,1.232160e+11,1.143010e+11,1.194370e+11,NaN
NetInterestIncome,NaN,NaN,-1.830000e+08,-1.060000e+08,1.980000e+08
InterestExpense,NaN,NaN,3.933000e+09,2.931000e+09,2.645000e+09


In [ ]:
class YahooIngestor:
    def __init__(self, tickers: list[str], models: list[str]):
        self.tickers = tickers
        self.models = models
    def ingest(self):
        for ticker in self.tickers:
            client = YahooClient(ticker)
            for model in self.models:
                raw_data = getattr(client, model)()
                if not raw_data.empty:
    
        

SyntaxError: incomplete input (2086912042.py, line 1)